In [1]:
# HMM smoother

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

from wrangle_updated import prep_test, trial, prep
from utilities import convert_to_words

In [ ]:
training_data = prep()

In [37]:
test_data1 = prep_test('test1_ymount_ycg.csv')
test_data4 = prep_test('GL_TEST1_CS.csv')
test_data5 = prep_test('GL_TEST2_CS.csv')
test_data6 = prep_test('GL_TEST3_CS_very_still.csv')

print training_data[0:1]
print test_data1[0:1]


Removed 2 NaN rows
Removed 2 NaN rows
Removed 1 NaN rows
Removed 3 NaN rows
   index  ACCEL_X   ACCEL_Y  ACCEL_Z    GYRO_X    GYRO_Y  GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  ...      std_gx   std_gy    std_gz  diff_x    diff_y    diff_z  diff_gx  diff_gy   diff_gz  state
0      0  -0.9562 -0.138667   0.3459  1.638267 -3.331367  2.6343            -0.959         -0.144333             0.343  ...    3.825866  3.18329  1.504622  -0.031 -0.082667 -0.086333  -16.768  -13.536 -5.874333      0

[1 rows x 29 columns]
   ACCEL_X  ACCEL_Y   ACCEL_Z  GYRO_X  GYRO_Y  GYRO_Z  rolling_median_x  rolling_median_y  rolling_median_z  rolling_max_x    ...        std_z   std_gx    std_gy    std_gz  diff_x  diff_y  diff_z  diff_gx  diff_gy   diff_gz
0  -0.9732  -0.2798  0.169633  0.5204  0.1606  0.6444         -0.975167         -0.279167          0.165333         -0.964    ...     0.013456  0.71578  3.435847  0.593049  -0.016  -0.013  -0.041   -2.866  -12.907 -2.276333

[1 rows x 27 col

In [4]:
pre_smooth = trial(training_data, test_data1)

[7 7 0 5 0 5 5 5 0 5 5 5 7 5 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
['OTHER', 'OTHER', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc']
Your Mount: 0.0975609756098
Your Side Control: 0.0
Your Close

In [39]:
pre_smooth2 = trial(training_data, test_data4)

[7 7 3 6 3 6 3 2 3 7 0 0 0 5 0 0 0 1 1 0 1 6 1 1 1 1 0 7 4 4 4 4 4 2 4 4 2
 3 2 2 2 4 2 5 0 0 5 5 5 5 0 5 5 5 2 4 4 4 4 4 4 4 4 4 4 4 4 4 6 6 4 6 6 6
 6 6 6 6 6 1 5]
['OTHER', 'OTHER', 'your_back_control', 'opponent_back_control', 'your_back_control', 'opponent_back_control', 'your_back_control', 'your_closed_guard', 'your_back_control', 'OTHER', 'your_mount', 'your_mount', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_mount', 'your_side_control', 'opponent_back_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_mount', 'OTHER', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'your_closed_guard', 'your_back_control', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_gu

In [40]:
pre_smooth3 = trial(training_data, test_data5)

[7 7 7 1 6 6 1 1 1 1 3 3 4 1 0 5 0 0 0 0 0 1 1 5 1 1 1 7 4 2 2 2 2 2 2 2 2
 2 2 2 7 0 0 5 0 5 5 5 5 5 5 5 5 7 2 4 4 4 4 4 2 2 4 4 2 4 4 4 4 4 4 3 6 4
 4 6 3 4 3 6 6 6 6 6 6 6 3 7 0 7]
['OTHER', 'OTHER', 'OTHER', 'your_side_control', 'opponent_back_control', 'opponent_back_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_back_control', 'your_back_control', 'opponent_mount_or_sc', 'your_side_control', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'opponent_closed_guard', 'your_side_control', 'your_side_control', 'your_side_control', 'OTHER', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'OTHER', 'your_mount', 'your_mount', 'oppon

In [41]:
pre_smooth4 = trial(training_data, test_data6)

[7 7 7 6 6 6 6 6 6 6 3 3 5 0 0 7 5 5 5 5 5 7 7 7 7 5 7 0 0 1 1 1 1 1 1 1 1
 1 1 1 0 2 2 2 2 2 7 7 7 2 2 2 2 2 7 2 2 7 7 7 7 2 2 2 0 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 6 4 4 4 4 4 7 7 7 7 7 7 7 7 7 2 4 4 4 4 4 4 4 4 7 4 4 4 4 7
 4 4 2 3 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 0 0 7 5]
['OTHER', 'OTHER', 'OTHER', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'opponent_back_control', 'your_back_control', 'your_back_control', 'opponent_closed_guard', 'your_mount', 'your_mount', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'OTHER', 'OTHER', 'OTHER', 'opponent_closed_guard', 'OTHER', 'your_mount', 'your_mount', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_side_control', 'your_

In [50]:
print pre_smooth
pre_smooth_words = convert_to_words(pre_smooth)
pre_smooth_words2 = convert_to_words(pre_smooth2)
pre_smooth_words3 = convert_to_words(pre_smooth3)
pre_smooth_words4 = convert_to_words(pre_smooth4)
print pre_smooth_words

[7 7 0 5 0 5 5 5 0 5 5 5 7 5 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
['OTHER', 'OTHER', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc', 'opponent_mount_or_sc']


In [6]:
n_components = 8 # ('ybc', 'ymount', 'ysc', 'ycg', 'ocg', 'osc_mount', 'obc', 'other')
# n_components = 3
startprob = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.65,]) # users will probably turn on sensor standing
# startprob = np.array([0.34, 0.33, 0.33])

In [16]:
# transmat = np.array([[0.34, 0.33, 0.33], [0.9, 0.05, 0.05], [0.9, 0.05, 0.05]])

"""
probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

transition_probability = {
        'ymt' : {'ymount': 0.800, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.800, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.800, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.800, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.100, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.800, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.800, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

transmat = np.array([
                    [0.800, 0.050, 0.010, 0.050, 0.001, 0.050, 0.001, 0.038], 
                    [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                    [0.010, 0.050, 0.800, 0.010, 0.050, 0.001, 0.001, 0.078], 
                    [0.050, 0.010, 0.050, 0.800, 0.001, 0.010, 0.001, 0.078],
                    [0.001, 0.050, 0.010, 0.050, 0.800, 0.050, 0.001, 0.038],
                    [0.100, 0.050, 0.010, 0.010, 0.001, 0.800, 0.001, 0.028],
                    [0.010, 0.050, 0.001, 0.010, 0.050, 0.001, 0.800, 0.078],
                    [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                    ])

In [54]:
# emissionprob = np.array([[0.34, 0.33, 0.33], [0.4, 0.55, 0.05], [0.05, 0.55, 0.4]])

"""

probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

emission_probability = {
        'ymt' : {'ymount': 0.600, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.250, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.600, 'ybc': 0.010, 'osc_mount': 0.250, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.700, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.101, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.210, 'ybc': 0.050, 'osc_mount': 0.600, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.300, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.500, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.110, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.700, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

emissionprob = np.array([
                        [0.600, 0.050, 0.010, 0.050, 0.001, 0.250, 0.001, 0.038], 
                        [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                        [0.010, 0.050, 0.600, 0.010, 0.250, 0.001, 0.001, 0.078], 
                        [0.050, 0.010, 0.050, 0.700, 0.001, 0.010, 0.101, 0.078],
                        [0.001, 0.050, 0.210, 0.050, 0.600, 0.050, 0.001, 0.038],
                        [0.300, 0.050, 0.010, 0.010, 0.001, 0.500, 0.001, 0.028],
                        [0.010, 0.050, 0.001, 0.110, 0.050, 0.001, 0.700, 0.078],
                        [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                        ])


In [55]:
# Hidden Markov Model with multinomial (discrete) emissions
model = hmm.MultinomialHMM(n_components=n_components,
                           n_iter=10,
                           verbose=False)

model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob
# model.n_features = 8

In [56]:
# observations = np.array([1, 1, 2, 2, 1, 0, 1, 2, 2, 0])
observations = np.array(pre_smooth)
observations2 = np.array(pre_smooth2)
observations3 = np.array(pre_smooth3)
observations4 = np.array(pre_smooth4)
a,b = model.sample(5)
print a,b
print '=========='

n_samples = len(observations)
data = observations.reshape((n_samples, -1))
print data

n_samples2 = len(observations2)
data2 = observations2.reshape((n_samples2, -1))

n_samples3 = len(observations3)
data3 = observations3.reshape((n_samples3, -1))

n_samples4 = len(observations4)
data4 = observations4.reshape((n_samples4, -1))

[[7]
 [7]
 [7]
 [7]
 [5]] [7 7 7 7 7]
[[7]
 [7]
 [0]
 [5]
 [0]
 [5]
 [5]
 [5]
 [0]
 [5]
 [5]
 [5]
 [7]
 [5]
 [5]
 [7]
 [5]
 [0]
 [6]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [4]
 [4]
 [4]
 [4]
 [4]
 [4]]


In [57]:
# decode(X, lengths=None, algorithm=None)[source]
# Find most likely state sequence corresponding to X.
# Will work best for organic tests

"""correct sequence
your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7


[3, 0, 1, 2, 5, 4, 6]

"""

"correct sequence\nyour_mount' if v == 0 \nelse 'your_side_control' if v == 1\nelse 'your_closed_guard' if v == 2\nelse 'your_back_control' if v == 3\nelse 'opponent_mount_or_sc' if v == 4\nelse 'opponent_closed_guard' if v == 5\nelse 'opponent_back_control' if v == 6\nelse 'OTHER' if v == 7\n\n\n[3, 0, 1, 2, 5, 4, 6]\n\n"

In [53]:


print 'TEST 1'

result = model.decode(data, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth)
print 'result accuracy {}%'.format(result[0])
print 'final result: {}'.format(result[1])

result_words = convert_to_words(result[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words)
print '====================='
print 'result words: {}'.format(result_words)

print '******************'
print 'TEST2'
result2 = model.decode(data2, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth2)
print 'result accuracy {}%'.format(result2[0])
print 'final result: {}'.format(result2[1])

result_words2 = convert_to_words(result2[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words2)
print '====================='
print 'result words: {}'.format(result_words2)

print '******************'
print 'TEST3'
result3 = model.decode(data3, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth3)
print 'result accuracy {}%'.format(result3[0])
print 'final result: {}'.format(result3[1])

result_words3 = convert_to_words(result3[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words3)
print '====================='
print 'result words: {}'.format(result_words3)

print '******************'
print 'TEST4'
result4 = model.decode(data4, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_smooth4)
print 'result accuracy {}%'.format(result4[0])
print 'final result: {}'.format(result4[1])

result_words4 = convert_to_words(result4[1])
print '====================='
print 'pre smooth words: {}'.format(pre_smooth_words4)
print '====================='
print 'result words: {}'.format(result_words4)

TEST 1
pre smooth: [7 7 0 5 0 5 5 5 0 5 5 5 7 5 5 7 5 0 6 4 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 4 4
 4 4 4 4]
result accuracy -56.3781316061%
final result: [7 7 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4
 4 4 4 4]
pre smooth words: ['OTHER', 'OTHER', 'your_mount', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'your_mount', 'opponent_back_control', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'your_closed_guard', 'opponent_mount_or_sc', 'opponent_mou